- Author: Benjamin Du
- Date: 2023-03-25 14:48:30
- Modified: 2023-03-26 16:17:03
- Title: Manipulate Pull Requests Using GitHub Rest APIs
- Slug: manipulate-pull-requests-using-github-REST-APIs
- Category: Computer Science
- Tags: Computer Science, programming, GitHub, Actions, REST, RESTful, API, Python, pull request

**Things on this page are fragmentary and immature notes/thoughts of the author. Please read with your own judgement!**

In [13]:
import json
import requests


def _build_headers(token: str) -> dict[str, str]:
    headers = {
        "Accept": "application/vnd.github+json",
    }
    if token:
        headers["Authorization"] = f"Bearer {token}"
    return headers


def list_pull_requests(token: str, owner: str, repo: str) -> list[dict]:
    resp = requests.get(
        url=f"https://api.github.com/repos/{owner}/{repo}/pulls",
        headers=_build_headers(token),
    )
    if not resp.ok:
        resp.raise_for_status()
    return resp.json()


def create_pull_request(token: str, owner: str, repo: str, data: dict[str, str]) -> dict:
    """Create a pull request.
    Notice that the error code 422 (Unprocessable Entity)
    means there's no changes to merge from the head branch to the base branch,
    so it is not really an error.
    :param token: The personal access token for authentication.
    :param owner: Owner of the repository.
    :param repo: Name of the repository.
    :param data: A dict containing information (e.g., base, head, title, body, etc.)
    about the pull request to be created.
    """
    if not ("head" in data and "base" in data):
        raise ValueError("The data dict must contains keys head and base!")
    resp = requests.post(
        url=f"https://api.github.com/repos/{owner}/{repo}/pulls",
        headers=_build_headers(token),
        data=json.dumps(data),
    )
    if resp.status_code == 422:
        err_msg = resp.json()["errors"][0]["message"]
        if "no commits between" in err_msg.lower():
            return None
        prs = list_pull_requests(token, OWNER, REPO)
        for pr in prs:
            if pr["head"]["ref"] == data["head"] and pr["base"]["ref"] == data["base"]:
                return pr
        raise RuntimeError("The existing PR is closed/merged before identified!")
    return resp.json()


def merge_pull_request(token: str, owner: str, repo: str, pr_number: int):
    resp = requests.put(
        url=f"https://api.github.com/repos/{owner}/{repo}/pulls/{pr_number}/merge",
        headers=_build_headers(token),
    )
    if not resp.ok:
        resp.raise_for_status()


def list_pull_request_files(token: str, owner, repo: str, pr_number: int):
    resp = requests.get(
        url=f"https://api.github.com/repos/{owner}/{repo}/pulls/{pr_number}/files",
        headers=_build_headers(token),
    )
    if not resp.ok:
        resp.raise_for_status
    return resp.json()

In [15]:
files = list_pull_request_files("", "legendu-net", "docker-rust-cicd", 1)

In [16]:
[file["filename"] for file in files]

['.github/workflows/create_pull_request.yml',
 '.github/workflows/pr.py',
 'Dockerfile',
 'build.sh']